In [32]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate

load_dotenv()
# Initialize Groq LLM
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.2
)

# Define a text prompt template
template = """
You are an OSINT AI classifier. 
Your task is to analyze a given TEXT and decide if it contains content related to an anti-India campaign.

Instructions:
1. Decide whether the text promotes, supports, or is connected to an anti-India campaign.
2. Give a confidence score between 0 (not related at all) and 1 (definitely related).
3. Explain briefly why you decided that.

Output strictly in JSON format only:
{{
  "is_anti_india": <true/false>,
  "confidence": <float between 0 and 1>,
  "reasoning": "<short reasoning>"
}}

TEXT TO ANALYZE:
"{query}"
"""

prompt = PromptTemplate(
    input_variables=["query"],
    template=template
)

def llm_block(query: str) -> str:
    """
    Send a query with a PromptTemplate to Groq LLM and return the response.
    """
    final_prompt = prompt.format(query=query)
    response = llm.invoke(final_prompt)
    return response.content

# Example usage

outtext = (llm_block("""
                Introduction
India and Maldives have shared diplomatic, defence, economic, and cultural relations for the past six decades. Located in a crucial geographical position in the Indian Ocean, Maldives is vital to India’s strategy for the Indian Ocean and its neighbourhood. For its part, Maldives reaps benefits from India’s economic assistance and net security provision. India has assisted Maldives in various ways since its independence in 1965, such as its pursuit of socio-economic development and modernisation, as well as maritime security. Their engagements flourished beginning in the late 1980s, when India launched ‘Operation Cactus’ to abort a coup in Maldives against Maumoon Abdul Gayoom’s autocratic regime.[1]

The cordial friendship continued throughout Maldives\’ first democratic government, elected in 2008.  However, with Abdulla Yameen coming to power in 2013, India-Maldives relations spiralled downward with his crackdown on democracy, proximity towards China, and anti-India rhetoric used to muster nationalist sentiments.  In 2018 a new president, Ibrahim Solih, was elected, who immediately worked to improve the relationship by initiating an ‘India First’ policy. The policy prioritised India for economic and defence partnerships, and showed greater sensitivity to Indian concerns emanating from Chinese investments and activities in Maldives.

Not everyone agrees with this policy, however. In October 2020, the opposition coalition—i.e., the Progressive Party of Maldives (PPM) and the People’s National Congress (PNC)— officially launched a challenge to the bilateral relationship through what it called the ‘India Out’ campaign. The campaign seeks to exploit anti-India sentiments, already prevalent parallel to the democratic transition and amidst allegations of India’s expansionist ambitions. ‘India Out’ aims to fuel more hatred by creating scepticism for India’s investments in Maldives, the defence partnerships between the two, and India’s net-security provisions. Both of the political parties behind the campaign are led by Yameen.

This paper traces the origins of ‘India Out’, its nature, and drivers, and its implications.
                """))
print(outtext)

{
  "is_anti_india": true,
  "confidence": 0.8,
  "reasoning": "The text mentions the 'India Out' campaign, which is explicitly anti-India, and aims to fuel hatred and scepticism towards India's investments, defence partnerships, and security provisions in Maldives. The campaign is led by political parties with a history of anti-India rhetoric and proximity to China."
}


In [19]:
from groq import Groq
import os

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
import base64

with open("anti-India.jpg", "rb") as f:
    img_base64 = base64.b64encode(f.read()).decode("utf-8")
    
    
# Define a text prompt template
template = """
You are an OSINT AI classifier. 
Your task is to analyze a given IMAGE and decide if it contains content related to an anti-India campaign.

Steps:
1. First, describe briefly what the image contains. 
2. Decide whether the image content promotes, supports, or is connected to an anti-India campaign.
3. Give a confidence score between 0 (not related at all) and 1 (definitely related).
4. Explain briefly why you decided that.

Output strictly in JSON format only:
{{
  "is_anti_india": <true/false>,
  "confidence": <float between 0 and 1>,
  "reasoning": "<short reasoning>"
}}

"""

prompt = PromptTemplate(
    template=template
)

completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", 
                 "text": template},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
            ]
        }
    ]
)


print(completion.choices[0].message)


ChatCompletionMessage(content='```\n{\n  "is_anti_india": true,\n  "confidence": 0.9,\n  "reasoning": "The image contains a sign that reads \'ASHAMED OF BEING INDIAN\', which directly expresses a negative sentiment towards being Indian. This suggests a strong connection to an anti-India campaign, as it reflects a critical or ashamed view of Indian identity or nationality."\n}\n```', role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None)


In [24]:
output = completion.choices[0].message.content
print(output)



```
{
  "is_anti_india": true,
  "confidence": 0.9,
  "reasoning": "The image contains a sign that reads 'ASHAMED OF BEING INDIAN', which directly expresses a negative sentiment towards being Indian. This suggests a strong connection to an anti-India campaign, as it reflects a critical or ashamed view of Indian identity or nationality."
}
```


In [36]:
import json
# Step 1: Strip markdown fences if present
clean_output = output.strip().strip("`")

# Step 2: Some LLMs return "json\n{...}", so handle that
if clean_output.startswith("json"):
    clean_output = clean_output[4:].strip()

# Step 3: Now parse
try:
    result = json.loads(clean_output)
    print(result)
    print("Confidence:", result["confidence"])
except json.JSONDecodeError as e:
    print("❌ JSON parsing error:", e)
    print("Raw output was:\n", clean_output)


{'is_anti_india': True, 'confidence': 0.9, 'reasoning': "The image contains a sign that reads 'ASHAMED OF BEING INDIAN', which directly expresses a negative sentiment towards being Indian. This suggests a strong connection to an anti-India campaign, as it reflects a critical or ashamed view of Indian identity or nationality."}
Confidence: 0.9


In [37]:
result_text = json.loads(outtext)
result_text

{'is_anti_india': True,
 'confidence': 0.8,
 'reasoning': "The text mentions the 'India Out' campaign, which is explicitly anti-India, and aims to fuel hatred and scepticism towards India's investments, defence partnerships, and security provisions in Maldives. The campaign is led by political parties with a history of anti-India rhetoric and proximity to China."}